Definimos una función genral que leerá los datos en el exámen y efectuará las transformaciones oportunas. 
Se definen primero las funciones por partes

In [2]:
# Función de lectura básica: irá dentro de la grande, al principio
def lectura(archivo):
    
    columnas_para_leer = ['loan_amnt', 'term', 'int_rate', 'grade', 'emp_length',
                          'home_ownership', 'annual_inc', 'loan_status', 'purpose', 'dti',
                          'pub_rec', 'application_type',
                          'annual_inc_joint', 'dti_joint', 'mort_acc', 'pub_rec_bankruptcies',
                          'last_fico_range_low', 'last_fico_range_high']
    
    

    data = pd.read_csv(archivo, 
                       skiprows=1, # saltar primera fila
                       skipfooter=2,  # saltar dos ultimas filas 
                       engine='python',
                       usecols=columnas_para_leer)
    return data

Función para la transformación de la variabe dependiente

In [4]:
def transformar_taget(data):
    no_current = (data.loan_status != 'Current')
    
    # Aplicacion del filtro
    data = data[no_current] 
    
    # Unificación en paga/no paga
    data.loan_status = data.loan_status.map(lambda x: 'Paga' if x == 'Fully Paid' else 'No paga')
    
    return data

Función para transformar la cuolumna del tipo de interés 

In [3]:
def transformar_tipo (data):
    data.int_rate = data.int_rate.str.rstrip('%').astype('float')
    return data

Funciones para la trasnformación de las columnas FICO

In [4]:
def rating_fico (x, y):
    if  x >= 800 & y <= 850:
        return 'Exceptional'
    elif x >= 749 & y <= 799:
        return 'Very good'
    elif x >= 670 & y <= 739:
        return 'Good'
    elif x >= 580 & y <= 669:
        return 'Fair'
    else:
        return 'Very poor'

In [5]:
def transformar_FICO(data):
    data['rating_fico'] = data.apply(lambda row: 
                                 rating_fico(row['last_fico_range_low'], 
                                             row['last_fico_range_high']), axis = 1)
    
    data.drop(['last_fico_range_low', 'last_fico_range_high'], axis=1, inplace=True)
    return data

Función para la transformación de la columna de bancarrotas

In [6]:
def transformar_bancarrotas(data):
    data.pub_rec_bankruptcies = data.pub_rec_bankruptcies.map(lambda x: '7 o menos' if x <= 7 else 'Más de 7')
    return data

Función para la transformación de la columna de registros públicos

In [7]:
def transformar_registro_publico(data):
    data.pub_rec = data.pub_rec.map(lambda x: '11 o menos' if x <= 11 else 'Más de 11')
    return data

Finalmente, se define una función general, que tomará como argumento los datos nuevos y aplicará todas estas funciones

In [5]:
def lectura_limpia (archivo):
    '''
    Esta función solo toma como parámetro el archivo csv que queremos leer, que ha de estar en formato
    csv. Comienza eliminando los duplicados y va aplicando todas las funciónes de transformación 
    previamente definidas
    
    '''
    # Lectura
    data = lectura(archivo)
    
    # Eliminar los duplicados 
    data = data.drop_duplicates()
    
    # Transformación del target
    data = transformar_taget(data)
    
    # Transformacion de tipo de interes 
    data = transformar_tipo(data)
    
    # Transformación columna FICO 
    data = transformar_FICO(data)
   
    # Transformación columna bancarrotas 
    data = transformar_bancarrotas(data)

    # Transformación columna registros publicos 
    data = transformar_registro_publico(data)
    
    return data